In [5]:
! sudo pip install MySQL-python

You are using pip version 7.0.3, however version 7.1.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 110kB 1.2MB/s 
  Running setup.py bdist_wheel for MySQL-python
  Stored in directory: /root/.cache/pip/wheels/8c/0d/11/d654cad764b92636ce047897dd2b9e1b0cd76c22f813c5851a
Successfully built MySQL-python


In [1]:
import requests
import MySQLdb as mdb
import pandas as pd
import numpy as np
import json
import ConfigParser
import time

In [2]:
propertiesFile = "my.properties"
cp = ConfigParser.ConfigParser()
cp.readfp(open(propertiesFile))

db_host     = cp.get('Params', 'db_host')
db_user     = cp.get('Params', 'db_user')
db_password = cp.get('Params', 'db_password')
db_db       = cp.get('Params', 'db_db')

In [3]:
conn = mdb.connect(host=db_host, user=db_user, passwd=db_password, db=db_db)

In [4]:
url1 = 'http://en.wikipedia.org/w/api.php?action=query&list=usercontribs&ucuser='
url2 = '&uclimit=5000&ucdir=newer&format=json'

In [5]:
cache = {}

In [6]:
query = "INSERT INTO contributions (comment, ns, pageid, parentid, revid, size, timestamp, title, editor_id) VALUES ('{}',{},{},{},{},{},'{}','{}',{})"
query2 = "UPDATE editor SET last_uccontinue='{}' where editor_id={}"

In [ ]:
query = "select * from editor"
df = pd.read_sql(query, conn)
cur = conn.cursor()
for r in range(df.shape[0]):
    row = df.iloc[r]
    name = row['name']
    editor_id = row['editor_id']
    uccontinue = row['last_uccontinue']
    url = url1 + name + url2
    if uccontinue > 0:
        url = url + '&uccontinue=' + str(uccontinue)
    try:
        data = cache[url]
    except KeyError:
        print 'Getting ' + url
        r = requests.get(url)
        data = json.loads(r.text)
        cache[url] = data
    contributions = data['query']['usercontribs']
    for contribution in contributions:
        title = contribution['title'].replace("'", "''")
        comment = contribution['comment'].replace("'", "''").encode('ascii', 'ignore')
        q = query.format(comment, contribution['ns'], contribution['pageid'], contribution['parentid'], contribution['revid'], contribution['size'], contribution['timestamp'], title, editor_id)
        cur.execute(q)
        conn.commit()
    if data.has_key('continue'):
        ucc = data['continue']['uccontinue']
        q2 = query2.format(ucc, editor_id)
        cur.execute(q2)
        conn.commit()
    else:
        ucc = -1

cur.close()

In [180]:
query = "select distinct pageid, title from contributions"
df2 = pd.read_sql(query, conn)

In [213]:
cache = {}

In [214]:
for r in range(df2.shape[0]):
    row = df2.iloc[r]
    title = row.title
    url = 'http://stats.grok.se/json/en/latest90/' + title
    try:
        data = cache[url]
    except KeyError:
        r = requests.get(url)
        data = json.loads(r.text)
        data['pageid'] = row.pageid
        cache[url] = data
        time.sleep(.5)

In [218]:
query = "INSERT INTO page_views (pageid, project, dt, views) VALUES({}, '{}', '{}', {}) ON DUPLICATE KEY UPDATE views=VALUES(views)"

In [222]:
cur = conn.cursor()
project = data['project']
pageid = data['pageid']
dvs = data['daily_views']
for dv in dvs.keys():
    pv = dvs[dv]
    q = query.format(pageid, project, dv, pv)
    cur.execute(q)
    conn.commit()

cur.close()

In [223]:
#http://stats.grok.se/json/en/latest90/apples
#http://dumps.wikimedia.org/other/pagecounts-raw/2015/2015-08/

In [ ]:
# List of gsow editors, last uccontinue

In [1]:

# List of pages edited
# By whom and when
# pageview data on all
# persist all

* get pageview data
* get pages changed by users in a list

* Define a metric measuring the significance of a change (e.g. include missing hyperlink vs. completely new article written)
* Track the number of page views for edited page of GSoW editors
* Provide exploratory and visual tools to assess and measure the impact of the team's efforts
